In [0]:
CREATE CATALOG IF NOT EXISTS usertraining;
CREATE SCHEMA IF NOT EXISTS usertraining.wikipedia;
USE usertraining.wikipedia;

In [0]:
SELECT * FROM PARQUET.`dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet`

In [0]:
CREATE OR REPLACE TABLE articles_bronze
USING DELTA -- optional
AS
SELECT * FROM PARQUET.`dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet`;

SELECT * FROM articles_bronze

In [0]:
DESCRIBE articles_bronze

In [0]:
CREATE SCHEMA IF NOT EXISTS usertraining.flights;
USE usertraining.flights;

In [0]:
create or replace temp view departuredelays as
  select * from csv.`dbfs:/databricks-datasets/flights/departuredelays.csv`;

select * from departuredelays

In [0]:
DROP TABLE IF EXISTS departuredelays_bronze;

CREATE TABLE departuredelays_bronze
USING DELTA -- optional
AS
  SELECT *
  FROM read_files("dbfs:/databricks-datasets/flights/departuredelays.csv",
    format => "csv",
    sep => ",",
    header => true,
    mode => "FAILFAST");

SELECT * FROM departuredelays_bronze

In [0]:
describe extended departuredelays_bronze

In [0]:
describe catalog usertraining

In [0]:
describe schema usertraining.flights

In [0]:
drop table if exists airportcodes_bronze;
create table airportcodes_bronze using delta;

In [0]:
copy into airportcodes_bronze
  FROM "dbfs:/databricks-datasets/flights/airport-codes-na.txt"
  FILEFORMAT = csv
  FORMAT_OPTIONS ("header" = "true", "delimiter" = "\t")
  COPY_OPTIONS ("mergeSchema" = "true");

In [0]:
select * from airportcodes_bronze

#EXTERNAL LOCATIONS

In [0]:
SHOW STORAGE CREDENTIALS

In [0]:
CREATE EXTERNAL LOCATION IF NOT EXISTS usertraining_ext
    URL 'abfss://external@adlanedbrmetastore01.dfs.core.windows.net'
    WITH (STORAGE CREDENTIAL `b80ec2c9-c62a-46e4-8380-e9d59ead1ed4-storage-credential-1746568360876`) --update from prev cell
    COMMENT 'User training external data'

In [0]:
SHOW EXTERNAL LOCATIONS

In [0]:
SHOW CREATE TABLE airportcodes_bronze

In [0]:
drop table if exists airportcodes_ext_bronze;

CREATE TABLE airportcodes_ext_bronze (
  City STRING,
  State STRING,
  Country STRING,
  IATA STRING)
USING csv
OPTIONS ("header" = "true", "delimiter" = "\t")
LOCATION 'abfss://external@adlanedbrmetastore01.dfs.core.windows.net/airportcodes';

insert into airportcodes_ext_bronze
  select * from airportcodes_bronze;

select * from airportcodes_ext_bronze;

#Built In Functions

In [0]:
DROP TABLE IF EXISTS departuredelays2_bronze;

CREATE TABLE departuredelays2_bronze;

COPY INTO departuredelays2_bronze FROM
  (SELECT *,
  CAST(CAST(CAST(date AS INT) AS TIMESTAMP) AS DATE) as date2,
  current_timestamp() AS updated,
  input_file_name() AS source_file
  FROM "dbfs:/databricks-datasets/flights/departuredelays.csv")
  FILEFORMAT = csv
  FORMAT_OPTIONS ("header" = "true", "delimiter" = ",")
  COPY_OPTIONS ("mergeSchema" = "true");

SELECT * FROM departuredelays2_bronze